<a href="https://colab.research.google.com/github/Craphtr/data-enginering-zoomcamp2025/blob/main/03-data-warehouse/dlt_workshop_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block" style = "background-color: black">,
        <p><b><font size="+4" color="orange">Homework Data Ingestion with dlt</font></b></p>
    </div>

In [ ]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator
import requests

#### __Q1.__

In [ ]:
!dlt --version

dlt 1.5.0


#### __Q2. Define and Run Pipeline on NYC Taxi Dataset__

In [ ]:
@dlt.resource(name="nyc_taxi_data")

def fetch_tripdata(service_type='yellow'):

    """Fetches NYC tripdata dynamically.
    - sevice_type: "yellow" or green"
    - Loops thru all years and months to fetch all available data
    """
    client = RESTClient(
        base_url = "https://us-central1-dlthub-analytics.cloudfunctions.net"
        #define pagination strategy - page-based pagination
        paginator = PageNumberPaginator(# pages numbered 1 to n
            base_page = 1, #start_page = 1
            total_path = None # No total count of pages known forehand, pagination ends when no more pages to process
        )
    )

    for page in client.paginate("data_enginnering_zoomcamp_api"): #API endpoint for retrieving data
        yield page #yielding page by page exposes you endpoint data and prevents memory overflow


#Define and run the dlt pipeline
pipeline = dlt.pipeline(destination="duckdb")
load_info = pipeline.run(fetch_nyc_taxi_data, write_disposition="replace")
print(load_info)



NameError: name 'filename' is not defined

#### __Connecting `duckdb` to a database__

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

#A database '<pipeline-duckdb>' was created in the current working directory to connect to it

Connect to the duckdb database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

#Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

#Describe the dataset
conn.sql("DESCRIBE").df()

In [ ]:
#Define the range of years from 2009 to 2024
start_year = 2009
end_year = 2024

#Define the rage of months
months = [f"{m:02d}" for m in range(1,13)]

#Loop through all years and months
for year in range(start_year, end_year):
    for month in months:
        file_name = f"{service_type}_tripdata_{year}-{month}.parquet"
        url = base_url + file_name

        print(f"Downloading {file_name}")
        response = requests.get(url)

        if response.status_code == 200:
            print(f"Successfully downloaded {file_name}")
        else:
            print(f"Failed to download {file_name}.status - {response.status_code}")
            continue


array([2009.])

#### __Q3. Explore the Dataset__

In [ ]:
df = pipeline.dataset(dataset_type="default").rides.df()

#### __Q4. Trip Duration Analysis__

In [ ]:
with pipeline.sql_client() as client():
  res = client_execute.sql(
      """
      SELECT
      AVG(date_diff('minute', 'trip_pickup_datetime', 'trip_dropoff_datetime'))
      FROM rides;
      """
  )
  print(res)